In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS
import joblib
import spacy

# Create your Flask app
app = Flask(__name__)

# Enable CORS for all routes in the app
CORS(app)  # This will allow requests from any domain to interact with the Flask server

slack_model = joblib.load('slack_issue_resolution_model.pkl')
nlp = spacy.load("en_core_web_lg")

nlp_ner = spacy.load("custom_ner_model")

@app.route("/slack/events", methods=["POST"])
def slack_events():
    data = request.json
    print("Received Slack event:", data)

        # Handle Slack URL verification
    if "type" in data and data["type"] == "url_verification":
        return jsonify({"challenge": data["challenge"]})

      # Process incoming messages
    if "event" in data:
        event = data["event"]
        
        # Ignore bot messages to prevent infinite loops
        if event.get("bot_id") is not None:
            return jsonify({"status": "ignored", "reason": "Bot message"})
        # Process incoming messages
        
        if "type" in event and event["type"] == "message" and "text" in event:
            user_message = event["text"]
            channel_id = event["channel"]
            thread_ts = event.get("thread_ts", event["ts"])  # Use the thread_ts if exists, else use the message's ts

            # Process message with NER model
            doc = nlp_ner(user_message)
            entities = {ent.label_: ent.text for ent in doc.ents}
            print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

            if "RAISE_INCIDENT" in entities and "SEVERITY" in entities:
                response_text = f"*Incident Raised:* Your incident has been successfully raised and marked as *{entities['SEVERITY']}* priority. :warning: \n\nThe *Site Reliability Engineering (SRE)* team will address it as soon as possible to ensure a swift resolution. :rocket:\n\nThank you for your patience!"
                send_message_to_slack(channel_id, response_text, thread_ts)
            else:
                # Preprocess message using SpaCy
                vectorized_message = nlp(user_message).vector.reshape(1, -1)
                # Predict issue category
                predicted_category = slack_model.predict(vectorized_message)[0]
                # Map category ID to resolution
                category_resolutions = {
                    0: "*Sorry, I couldn't determine the issue:* I'm unable to determine the issue from your message. Could you please provide more details or clarify the problem? :thinking_face: If needed, I can escalate this to a support team member for further assistance. :telephone_receiver:",
                    1: "*Offer Not Displayed:* The offer is not displayed because the customer has opted out of solicitation, making them ineligible for any offers. :x: However, if you are not satisfied with this response, I can escalate the issue by raising an incident so a support representative can review it further. Would you like me to raise it? :raised_hands:",
                    2: "*Access Issue:* The Customer Care Professional doesn't have the required access to manage the offer. To enable the user to service the offer, an access request needs to be raised. :lock: Would you like me to initiate the request for you? :bar_chart:"
                }
                resolution = category_resolutions.get(predicted_category, "No resolution found.")
                # Send response back to Slack
                send_message_to_slack(channel_id, resolution, thread_ts)
        return jsonify({"status": "error", "message": "Event processed"})

# Function to send message back to Slack
def send_message_to_slack(channel, text, thread_ts):
    import requests
    SLACK_BOT_TOKEN = ""  # Replace with your bot token
    url = "https://slack.com/api/chat.postMessage"
    headers = {"Authorization": f"Bearer {SLACK_BOT_TOKEN}", "Content-Type": "application/json"}
    data = {"channel": channel, "text": text, "thread_ts": thread_ts, "username": "ResolveBotAI", "icon_emoji": ":robot_face:"}
    requests.post(url, headers=headers, json=data)
        

if __name__ == "__main__":
    app.run(port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


Received Slack event: {'token': 'vcOktFRX27KKXwM6mDwo1R9u', 'team_id': 'T08GLQ3HC07', 'context_team_id': 'T08GLQ3HC07', 'context_enterprise_id': 'None', 'api_app_id': 'A08H0C26BEZ', 'event': {'user': 'U08GLQ3JKGX', 'type': 'message', 'ts': '1741635490.564199', 'client_msg_id': 'b33e1972-15c5-4b71-a966-641b48be624e', 'text': 'raise an incident with high severity', 'team': 'T08GLQ3HC07', 'blocks': [{'type': 'rich_text', 'block_id': 'rBoqc', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'test from vg ops'}]}]}], 'channel': 'C08H54N6B6G', 'event_ts': '1741635490.564199', 'channel_type': 'channel'}, 'type': 'event_callback', 'event_id': 'Ev08H62YCTEY', 'event_time': 1741635490, 'authorizations': [{'enterprise_id': 'None', 'team_id': 'T08GLQ3HC07', 'user_id': 'U08H42J6JQ3', 'is_bot': 'True', 'is_enterprise_install': 'False'}], 'is_ext_shared_channel': 'False', 'event_context': '4-eyJldCI6Im1lc3NhZ2UiLCJ0aWQiOiJUMDhHTFEzSEMwNyIsImFpZCI6IkEwOEgwQzI2QkVaIiwiY2

127.0.0.1 - - [11/Mar/2025 16:07:05] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2025 16:07:48] "POST /slack/events HTTP/1.1" 200 -


Received Slack event: {'token': 'vcOktFRX27KKXwM6mDwo1R9u', 'challenge': 's42S2dGPzP5ynlNSUWysdiBpoCzg3iGk7ofwh8pJXhtWAYxpEwyO', 'type': 'url_verification'}
Received Slack event: {'token': 'vcOktFRX27KKXwM6mDwo1R9u', 'team_id': 'T08GLQ3HC07', 'context_team_id': 'T08GLQ3HC07', 'context_enterprise_id': None, 'api_app_id': 'A08H0C26BEZ', 'event': {'user': 'U08GLQ3JKGX', 'type': 'message', 'ts': '1741723678.218029', 'client_msg_id': '6b6b0f03-0b9a-402c-889c-9bd969ed3ae0', 'text': 'Offer not loading', 'team': 'T08GLQ3HC07', 'blocks': [{'type': 'rich_text', 'block_id': 'eBuex', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Offer not loading'}]}]}], 'channel': 'C08H54N6B6G', 'event_ts': '1741723678.218029', 'channel_type': 'channel'}, 'type': 'event_callback', 'event_id': 'Ev08HL612517', 'event_time': 1741723678, 'authorizations': [{'enterprise_id': None, 'team_id': 'T08GLQ3HC07', 'user_id': 'U08H42J6JQ3', 'is_bot': True, 'is_enterprise_install': False}], '

...

Received Slack event: {'token': 'vcOktFRX27KKXwM6mDwo1R9u', 'team_id': 'T08GLQ3HC07', 'context_team_id': 'T08GLQ3HC07', 'context_enterprise_id': None, 'api_app_id': 'A08H0C26BEZ', 'event': {'user': 'U08H42J6JQ3', 'type': 'message', 'ts': '1741728721.362319', 'bot_id': 'B08H42J4LGK', 'app_id': 'A08H0C26BEZ', 'text': '*Offer Not Displayed:* The offer is not displayed because the customer has opted out of solicitation, making them ineligible for any offers. :x: However, if you are not satisfied with this response, I can escalate the issue by raising an incident so a support representative can review it further. Would you like me to raise it? :raised_hands:', 'team': 'T08GLQ3HC07', 'bot_profile': {'id': 'B08H42J4LGK', 'deleted': False, 'name': 'ResolveBotAI', 'updated': 1741727901, 'app_id': 'A08H0C26BEZ', 'icons': {'image_36': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c606919fdf38cbda995_36.png', 'image_48': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c606919

127.0.0.1 - - [11/Mar/2025 17:32:18] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2025 17:32:19] "POST /slack/events HTTP/1.1" 200 -


Received Slack event: {'token': 'vcOktFRX27KKXwM6mDwo1R9u', 'team_id': 'T08GLQ3HC07', 'context_team_id': 'T08GLQ3HC07', 'context_enterprise_id': None, 'api_app_id': 'A08H0C26BEZ', 'event': {'user': 'U08H42J6JQ3', 'type': 'message', 'ts': '1741728737.892799', 'bot_id': 'B08H42J4LGK', 'app_id': 'A08H0C26BEZ', 'text': "*Access Issue:* The Customer Care Professional doesn't have the required access to manage the offer. To enable the user to service the offer, an access request needs to be raised. :lock: Would you like me to initiate the request for you? :bar_chart:", 'team': 'T08GLQ3HC07', 'bot_profile': {'id': 'B08H42J4LGK', 'deleted': False, 'name': 'ResolveBotAI', 'updated': 1741727901, 'app_id': 'A08H0C26BEZ', 'icons': {'image_36': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c606919fdf38cbda995_36.png', 'image_48': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c606919fdf38cbda995_48.png', 'image_72': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c6

127.0.0.1 - - [11/Mar/2025 17:32:35] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2025 17:32:36] "POST /slack/events HTTP/1.1" 200 -


Received Slack event: {'token': 'vcOktFRX27KKXwM6mDwo1R9u', 'team_id': 'T08GLQ3HC07', 'context_team_id': 'T08GLQ3HC07', 'context_enterprise_id': None, 'api_app_id': 'A08H0C26BEZ', 'event': {'user': 'U08H42J6JQ3', 'type': 'message', 'ts': '1741728754.702179', 'bot_id': 'B08H42J4LGK', 'app_id': 'A08H0C26BEZ', 'text': "*Sorry, I couldn't determine the issue:* I'm unable to determine the issue from your message. Could you please provide more details or clarify the problem? :thinking_face: If needed, I can escalate this to a support team member for further assistance. :telephone_receiver:", 'team': 'T08GLQ3HC07', 'bot_profile': {'id': 'B08H42J4LGK', 'deleted': False, 'name': 'ResolveBotAI', 'updated': 1741727901, 'app_id': 'A08H0C26BEZ', 'icons': {'image_36': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c606919fdf38cbda995_36.png', 'image_48': 'https://avatars.slack-edge.com/2025-03-11/8589372074499_9c606919fdf38cbda995_48.png', 'image_72': 'https://avatars.slack-edge.com/2025-